# Read Data

In [50]:
! pip install -q kaggle
! mkdir ~/.kaggle

# Menyalin berkas kaggle.json pada direktori aktif saat ini ke folder .kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [51]:
! kaggle datasets download -d ruchi798/bookcrossing-dataset

bookcrossing-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [52]:
# Mengekstrak berkas zip ke direktori aktif saat ini
!unzip /content/bookcrossing-dataset.zip

Archive:  /content/bookcrossing-dataset.zip
replace Book reviews/Book reviews/BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Book reviews/Book reviews/BX-Book-Ratings.csv  
replace Book reviews/Book reviews/BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Book reviews/Book reviews/BX-Users.csv  
replace Book reviews/Book reviews/BX_Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Book reviews/Book reviews/BX_Books.csv  
replace Books Data with Category Language and Summary/Preprocessed_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Books Data with Category Language and Summary/Preprocessed_data.csv  y
y
y



In [53]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

# Data Understanding

In [54]:
path = "/content/Book reviews/Book reviews/"
df_user = pd.read_csv(path+'BX-Users.csv', sep=";", error_bad_lines=False,encoding='latin-1')

df_books = pd.read_csv(path+'BX_Books.csv', sep=";", error_bad_lines=False,encoding='latin-1')

df_rating = pd.read_csv(path+'BX-Book-Ratings.csv', sep=";", error_bad_lines=False,encoding='latin-1')

<ipython-input-54-dfd0fca07254>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_user = pd.read_csv(path+'BX-Users.csv', sep=";", error_bad_lines=False,encoding='latin-1')
<ipython-input-54-dfd0fca07254>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_books = pd.read_csv(path+'BX_Books.csv', sep=";", error_bad_lines=False,encoding='latin-1')
<ipython-input-54-dfd0fca07254>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_rating = pd.read_csv(path+'BX-Book-Ratings.csv', sep=";", error_bad_lines=False,encoding='latin-1')


# Exploratory Data Analysis (EDA)

In [55]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   User-ID   278858 non-null  int64  
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [56]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271379 non-null  object
 1   Book-Title           271379 non-null  object
 2   Book-Author          271378 non-null  object
 3   Year-Of-Publication  271379 non-null  int64 
 4   Publisher            271377 non-null  object
 5   Image-URL-S          271379 non-null  object
 6   Image-URL-M          271379 non-null  object
 7   Image-URL-L          271379 non-null  object
dtypes: int64(1), object(7)
memory usage: 16.6+ MB


In [57]:
df_rating.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   User-ID      1149780 non-null  int64 
 1   ISBN         1149780 non-null  object
 2   Book-Rating  1149780 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [58]:
df_books = df_books.drop(['Year-Of-Publication', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1)
rating_book = df_books.merge(df_rating, on='ISBN')
rating_book.head()

,ISBN,Book-Title,Book-Author,Publisher,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,Oxford University Press,2,0
1,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,8,5
2,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,11400,0
3,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,11676,8
4,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,41385,0


|index| ISBN      |	Book-Title              |	Book-Author|	Publisher       |	User-ID|	Book-Rating|	Number of Book-Rating_x|	Number of Book-Rating_y|
|-----|-----------|-------------------------|------------|-----------       |--------|-------------|-------------------------|-------------------------|
|0    |	0399135782|	The Kitchen God's Wife	|Amy Tan     |	Putnam Pub Group|	8      |	0          |	33                     |	33                     |
|1    |	0399135782|	The Kitchen God's Wife	|Amy Tan     |	Putnam Pub Group|	11676  |	9          |	33                     |	33                     |
|2    |	0399135782|	The Kitchen God's Wife	|Amy Tan     |	Putnam Pub Group|	29526  |	9          |	33                     |	33                     |
|3    |	0399135782|	The Kitchen God's Wife	|Amy Tan     |	Putnam Pub Group|	36836  |	0          |	33                     |	33                     |
|4    |	0399135782|	The Kitchen God's Wife	|Amy Tan     |	Putnam Pub Group|	46398  |	9          |	33                     |	33                     |

### Tambahkan jumlah peringkat yang ditetapkan. Hapus dari pengguna yang memberi peringkat beberapa kali


In [59]:
number_rating = rating_book.groupby('ISBN')['Book-Rating'].count().reset_index()
number_rating.rename(columns={'Book-Rating': 'Number of Book-Rating'}, inplace=True)
rating_book = rating_book.merge(number_rating, on='ISBN')
rating_book

,ISBN,Book-Title,Book-Author,Publisher,User-ID,Book-Rating,Number of Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,Oxford University Press,2,0,1
1,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,8,5,14
2,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,11400,0,14
3,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,11676,8,14
4,0002005018,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,41385,0,14
...,...,...,...,...,...,...,...
1031170,0440400988,There's a Bat in Bunk Five,Paula Danziger,Random House Childrens Pub (Mm),276463,7,1
1031171,0525447644,From One to One Hundred,Teri Sloat,Dutton Books,276579,4,1
1031172,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,HarperSanFrancisco,276680,0,1
1031173,0192126040,Republic (World's Classics),Plato,Oxford University Press,276680,0,1


###  memfilter rating_book diatas 30

In [60]:
rating_book = rating_book[rating_book['Number of Book-Rating'] >= 30]
rating_book.shape

(315065, 7)

### menghapus duplikat data


In [61]:
rating_book.drop_duplicates(['User-ID', 'Book-Title'], inplace=True)
rating_book.shape

<ipython-input-61-487ae7e8e22e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_book.drop_duplicates(['User-ID', 'Book-Title'], inplace=True)


(313546, 7)

### membuat table pivot

In [62]:
rating_book_pivot = rating_book.pivot_table(columns='User-ID', index='Book-Title', values='Book-Rating')
rating_book_pivot.fillna(0, inplace=True)
rating_book_pivot

User-ID,8,9,10,14,16,17,26,32,39,42,...,278831,278832,278836,278838,278843,278844,278846,278849,278851,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16 Lighthouse Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zoya,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\O\"" Is for Outlaw""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Modeling

In [63]:
book_matrix = csr_matrix(rating_book_pivot)
model = NearestNeighbors(algorithm='brute')
model.fit(book_matrix)

NearestNeighbors(algorithm='brute')

In [64]:
rating_book_pivot.iloc[60, :].values.reshape(1,-1)

array([[0., 0., 0., ..., 0., 0., 0.]])

In [65]:
_, recommendations = model.kneighbors(rating_book_pivot.iloc[60, :].values.reshape(1,-1))
recommendations

array([[  60, 1180, 2622, 1181, 2845]])

In [66]:
for i in range(len(recommendations)):
    print(recommendations[i])
    print(rating_book_pivot.index[recommendations[i]])

[  60 1180 2622 1181 2845]
Index(['A Heartbreaking Work Of Staggering Genius : A Memoir Based on a True Story',
       'Golden Cup', 'Tall, Dark, and Deadly', 'Golden Orange',
       'The Curse of the Mummy's Tomb (Goosebumps, No 5)'],
      dtype='object', name='Book-Title')


In [67]:
def recomend_book(book_name):
    book_id = np.where(rating_book_pivot.index == book_name)[0][0]
    _, recommendations = model.kneighbors(rating_book_pivot.iloc[book_id,:].values.reshape(1,-1))
    for i in range(len(recommendations)):
        if i == 0:
            print(f"For book \"{book_name}\" is recommended")
        if not i:
            print(rating_book_pivot.index[recommendations[i]])

In [68]:
recomend_book('Fatal Terrain')

For book "Fatal Terrain" is recommended
Index(['Fatal Terrain', 'Golden Cup', 'GEMINI CONTENDERS',
       'Hidden Leaves (Debeers)', 'Tall, Dark, and Deadly'],
      dtype='object', name='Book-Title')


# Evaluation


In [69]:
! pip install surprise
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
from surprise import SVD


In [70]:
rating_book.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 313546 entries, 30 to 790969
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   ISBN                   313546 non-null  object
 1   Book-Title             313546 non-null  object
 2   Book-Author            313546 non-null  object
 3   Publisher              313546 non-null  object
 4   User-ID                313546 non-null  int64 
 5   Book-Rating            313546 non-null  int64 
 6   Number of Book-Rating  313546 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 19.1+ MB


In [71]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df (rating_book[['User-ID','Number of Book-Rating', 'Book-Rating']],reader)
model = SVD()
results = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5)

print("Mean RMSE:", results['test_rmse'].mean())
print("Mean MAE:", results['test_mae'].mean())

Mean RMSE: 3.740682932553402
Mean MAE: 3.2001200612136103
